In [10]:
# importing all required packages & notebook extensions at the start of the notebook
import os
import pandas as pd
import qiime2 as q2
from qiime2 import Visualization

%matplotlib inline

or_dir = '../data' #original data (demux sequences, metadata)
data_dir = 'data' #data from polybox (ASV, taxonomy analysis)

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

In [6]:
! wget -nv -O data.zip 'https://polybox.ethz.ch/index.php/s/pNA39R0rl2xMMj9/download'
! unzip -q data.zip #-d $data_dir
! mv data data2
! mv data2/taxonomy/data .
! cp data2/ASV/data/* data
! rm -r data2
! rm data.zip

2022-10-25 13:00:58 URL:https://polybox.ethz.ch/index.php/s/pNA39R0rl2xMMj9/download [894620633] -> "data.zip" [1]


In [13]:
! pwd

/home/jovyan/HelicopterBaby/Phylogeny, alpha and beta diversity


Q: should we use filtered table or not???

In [16]:
! qiime feature-table summarize \
  --i-table $data_dir/PJNB_dada2_table_.qza \
  --m-sample-metadata-file $or_dir/metadata.tsv \
  --o-visualization $data_dir/feature-table.qzv

Saved Visualization to: data/feature-table.qzv


In [17]:
Visualization.load(f'{data_dir}/feature-table.qzv')

<visualization: Visualization uuid: dcebd226-ee54-421d-8d9a-9dcee077c86c>

In [ ]:
! qiime diversity alpha-rarefaction \
    --i-table $data_dir/feature-table.qza \
    --i-phylogeny $data_dir/insertion-tree.qza \
    --p-max-depth 10000 \
    --m-metadata-file $data_dir/metadata_proc.tsv \
    --o-visualization $data_dir/alpha-rarefaction.qzv

In [ ]:
Visualization.load(f'{data_dir}/alpha-rarefaction.qzv')

In [ ]:
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/feature-table.qza \
  --i-phylogeny $data_dir/insertion-tree.qza \
  --m-metadata-file $data_dir/metadata_proc.tsv \
  --p-sampling-depth 1500 \
  --output-dir $data_dir/core-metrics-results

In [ ]:
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file $data_dir/metadata_proc.tsv \
  --o-visualization $data_dir/core-metrics-results/faith-pd-group-significance.qzv

In [ ]:
Visualization.load(f'{data_dir}/core-metrics-results/faith-pd-group-significance.qzv')

In [ ]:
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file $data_dir/metadata_proc.tsv \
  --o-visualization $data_dir/core-metrics-results/faith-pd-group-significance-numeric.qzv

In [ ]:
Visualization.load(f'{data_dir}/core-metrics-results/faith-pd-group-significance-numeric.qzv')